In [1]:
!pip install colorama
!pip install -U tensorly
!pip install -U tensorly-torch

     |████████████████████████████████| 198 kB 8.0 MB/s 
     |████████████████████████████████| 154 kB 71.8 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab-Notebooks/tensor_adapt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab-Notebooks/tensor_adapt


In [3]:
%%shell
export PYTHONPATH=$PYTHONPATH:$(pwd)

CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/Colab-Notebooks/tensor_adapt/main.py --tensor --shared layer2 --rotation_type expand \
			--group_norm 8 \
			--nepoch 150 --milestone_1 75 --milestone_2 125 \
			--batch_size 64 \
			--outf results/cifar10_tensor_layer2_gn_expand

Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab-Notebooks/tensor_adapt/main.py", line 8, in <module>
    from utils.misc import *
  File "/content/drive/MyDrive/Colab-Notebooks/tensor_adapt/utils/misc.py", line 3, in <module>
    from colorama import Fore
ModuleNotFoundError: No module named 'colorama'


CalledProcessError: ignored